In [1]:
%load_ext autoreload
%autoreload 2

In [24]:
cm = torch.cat([I,J])
print(cm)
print(cm.unsqueeze(0))
#torch.nn.functional.cosine_similarity(cm.unsqueeze(0),cm.unsqueeze(1))
torch.nn.functional.cosine_similarity(cm.unsqueeze(0), cm.unsqueeze(1), dim = 2)

tensor([[ 1.0000,  2.0000],
        [ 3.0000, -2.0000],
        [ 1.0000,  5.0000],
        [ 1.0000,  0.7500],
        [ 2.8000, -1.7500],
        [ 1.0000,  4.7000]])
tensor([[[ 1.0000,  2.0000],
         [ 3.0000, -2.0000],
         [ 1.0000,  5.0000],
         [ 1.0000,  0.7500],
         [ 2.8000, -1.7500],
         [ 1.0000,  4.7000]]])


tensor([[ 1.0000, -0.1240,  0.9648,  0.8944, -0.0948,  0.9679],
        [-0.1240,  1.0000, -0.3807,  0.3328,  0.9996, -0.3694],
        [ 0.9648, -0.3807,  1.0000,  0.7452, -0.3534,  0.9999],
        [ 0.8944,  0.3328,  0.7452,  1.0000,  0.3604,  0.7533],
        [-0.0948,  0.9996, -0.3534,  0.3604,  1.0000, -0.3419],
        [ 0.9679, -0.3694,  0.9999,  0.7533, -0.3419,  1.0000]])